In [1]:
import json
import math
import pandas as pd

#from text_extraction import file_finder, TextExtraction
from data_creator import create_data
from utils import get_context, few_shot_cot, get_fewshot_cot_examples, get_label
from dotenv import load_dotenv
from pathlib import Path

from sklearn.metrics import f1_score
from tqdm import tqdm

In [2]:
df_dict = create_data(500, 30)

def filter_label(dataframes_dict: dict[pd.DataFrame], label: int) -> pd.DataFrame:
    # Create an empty list to store filtered DataFrames
    filtered_dataframes = []
    
    for key, df in dataframes_dict.items():
        if 'Label' in df.columns:
            filtered_df = df[df['Label'] == label]
            
            filtered_dataframes.append(filtered_df)
    
    result_df = pd.concat(filtered_dataframes, ignore_index=True)
    
    return result_df

def sample_data(dataframe):
    sampled_df = dataframe.sample(n=100, random_state=42) 
    return sampled_df



opinionated_data = filter_label(df_dict, 1)
neutral_data = sample_data(filter_label(df_dict, 0))

In [3]:
df = pd.concat([opinionated_data, neutral_data], ignore_index=True)

In [4]:
def get_precictions(df):
    predictions = []
    for i in tqdm(range(len(df))):
        #name = df["Authors"].iloc[i]
        title = df["Title"].iloc[i]
        context = get_context(df["context"].iloc[i])
        footnote = df["footnote_text"].iloc[i]
        #pred = context_sentiment(context)
        pred = few_shot_cot(examples = get_fewshot_cot_examples(), citation=context, title=title, footnote=footnote)
        
        #while pred != "0" and pred != "1":
            #print("Retrying prediction...")
            #pred = few_shot_cot(examples = get_fewshot_cot_examples(), citation=context, title=title, footnote=footnote)
        pred = get_label(pred)
        predictions.append(pred)
        #predictions = [int(i) for i in predictions]           
    return predictions

predictions = get_precictions(df)

100%|██████████| 199/199 [10:50<00:00,  3.27s/it]


In [6]:
labels = df["Label"].tolist()

def calculate_accuracy_per_label(predictions, labels, label_value):
    # Create a boolean array indicating whether the label matches the specified value
    label_matches = [l == label_value for l in labels]
  
    # Extract predictions for instances where the label matches the specified value
    matched_predictions = [p for i, p in enumerate(predictions) if label_matches[i]]
        
    return sum(matched_predictions)/100 if label_value == 1 else (len(matched_predictions) - sum(matched_predictions))/100
   
f1 = f1_score(predictions, labels)
accuracy_label_0 = calculate_accuracy_per_label(predictions, labels, label_value=0)
accuracy_label_1 = calculate_accuracy_per_label(predictions, labels, label_value=1)

print("F1 score:", f1)
print("Accuracy for label 0:", accuracy_label_0)
print("Accuracy for label 1:", accuracy_label_1)

F1 score: 0.6282722513089004
Accuracy for label 0: 0.68
Accuracy for label 1: 0.6
